# Pip Installs

In [1]:
%%capture 
%pip install datasets -q
%pip install openai -q
%pip install tiktoken -q
%pip install langchain -q
%pip install scipy -q
%pip install ragatouille -q
%pip install --upgrade jupyter ipywidgets -q
%pip install aiohttp nest_asyncio -q
%pip install asyncio -q
%pip install -U sentence-transformers -q
%pip install dotenv -q
%pip install ast -q
%pip install plotly -q

# Import Packages

In [2]:
import ast  # for converting embeddings saved as strings back to arrays
import openai
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os
from scipy import spatial
import ipywidgets
from datasets import load_dataset
import re  # for cutting <ref> links out of Wikipedia articles
from tqdm.notebook import tqdm

# Chunking Text
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

# Reranking
from ragatouille import RAGPretrainedModel

from typing import Optional

# Asynchronous requests
import aiohttp
import asyncio
from tqdm.asyncio import tqdm as atqdm

# Embedded chuck 
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import ast


# **Experiment: GPT Model**

In [3]:
GPT_MODEL = "gpt-3.5-turbo" # Reader model

# Files
OUTPUT_FILE_NAME = "./../csv_files/expOutputs/experiment_baseline_3.5.csv"


In [4]:
load_dotenv()
client = openai.OpenAI(api_key= os.getenv('OPENAI_API_KEY'))

In [5]:
def ask(query: str, model: str = GPT_MODEL) -> str:
    """Sends a query to the OpenAI API and returns the response."""
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": query},
    ]

    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )

    return response.choices[0].message.content

# Inferencing

In [6]:
def process_prompts(df, num_prompts):
    """Process prompts from the dataframe and get responses from the API."""
    results = []
    
    for i in tqdm(range(min(num_prompts, len(df))), desc="Processing prompts"):
        current_question = df.loc[i, 'Question']
        print(f"\nProcessing prompt number: {i + 1}")
        print(f"Current prompt: {current_question}")
        
        answer = ask(current_question)
        
        results.append({
            'Question': current_question,
            'Answer': answer
        })
        
        print(f"\nAnswer:")
        print(f"{'-'*30}")
        print(f"{answer}")
        print(f"{'-'*30}")
    
    return pd.DataFrame(results)

In [7]:
df = pd.read_csv('./../csv_files/rag_questions.csv', low_memory=False)

results_df = process_prompts(df, len(df))

results_df.to_csv(OUTPUT_FILE_NAME,  encoding='utf-8', index=False)

Processing prompts:   0%|          | 0/56 [00:00<?, ?it/s]


Processing prompt number: 1
Current prompt: I am using a transmitter to be used for WiFi operation. This transmitter is going to transmit a peak power of 20 dBm on both 20 Mhz and 40 MHz bandwidth. This transmitter can hop on any frequency from 2412 MHz to 2484 MHz. I am going to use an antenna which has a peak gain of 0.5 dBi. The antenna operates on the 2450 MHz band with a 100 MHz bandwidth. Does this transmitter follow FCC regulations? 

Answer:
------------------------------
To determine if the transmitter follows FCC regulations, we need to consider a few key points:

1. Transmit Power: The transmitter is transmitting at a peak power of 20 dBm, which is equivalent to 100 mW. This power level is within the FCC limits for unlicensed operation in the 2.4 GHz band.

2. Bandwidth: The transmitter is operating on both 20 MHz and 40 MHz bandwidths, which are common WiFi channel widths. These bandwidths are also within the FCC regulations for WiFi operation.

3. Frequency Range: The tra

In [8]:
import platform

def send_notification(title, message):
    system = platform.system()
    if system == "Darwin":  # macOS
        os.system(f"osascript -e 'display notification \"{message}\" with title \"{title}\"'")
        os.system("afplay /System/Library/Sounds/Glass.aiff")  # Play notification sound
    elif system == "Linux":
        os.system(f'notify-send "{title}" "{message}"')
        os.system("paplay /usr/share/sounds/freedesktop/stereo/complete.oga")  # Play notification sound
    elif system == "Windows":
        from plyer import notification
        notification.notify(
            title=title,
            message=message,
            timeout=10  # Notification duration in seconds
        )
        import winsound
        winsound.MessageBeep(winsound.MB_ICONASTERISK)  # Play notification sound
    else:
        print(f"Notification: {title} - {message}")

send_notification(
    title="Experiment Finished",
    message=f"Your {GPT_MODEL} baseline experiment has finished running.",
)


NameError: name 'platform' is not defined

In [ ]:
ask('I am using a transmitter to be used for WiFi operation. This transmitter is going to transmit a peak power of 20 dBm with 20 MHz bandwidth. This transmitter would have a center frequency of 1400 MHz. I am going to use an antenna which has a peak gain of 0.5 dBi. The antenna operates on the 2450 MHz band with a 100 MHz bandwidth. Does this transmitter follow FCC regulations? ')

In [ ]:
ask('I am using a transmitter to be used for WiFi operation. This transmitter is going to transmit a peak power of 40 dBm on both 20 Mhz and 40 MHz bandwidth. This transmitter can hop on any frequency from 2412 MHz to 2484 MHz. I am going to use an antenna which has a peak gain of 0.5 dBi. The antenna operates on the 2450 MHz band with a 100 MHz bandwidth. Does this transmitter follow FCC regulations? ')